*Note: You are currently reading this using Google Colaboratory which is a cloud-hosted version of Jupyter Notebook. This is a document containing both text cells for documentation and runnable code cells. If you are unfamiliar with Jupyter Notebook, watch this 3-minute introduction before starting this challenge: https://www.youtube.com/watch?v=inN8seMm7UI*

---

In this challenge, you need to create a machine learning model that will classify SMS messages as either "ham" or "spam". A "ham" message is a normal message sent by a friend. A "spam" message is an advertisement or a message sent by a company.

You should create a function called `predict_message` that takes a message string as an argument and returns a list. The first element in the list should be a number between zero and one that indicates the likeliness of "ham" (0) or "spam" (1). The second element in the list should be the word "ham" or "spam", depending on which is most likely.

For this challenge, you will use the [SMS Spam Collection dataset](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/). The dataset has already been grouped into train data and test data.

The first two cells import the libraries and data. The final cell tests your model and function. Add your code in between these cells.


In [1]:
# import libraries
try:
  # %tensorflow_version only exists in Colab.
  !pip install tf-nightly
except Exception:
  pass
import tensorflow as tf
import pandas as pd
from tensorflow import keras
!pip install tensorflow-datasets
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt

print(tf.__version__)

     |████████████████████████████████| 503.2 MB 4.9 kB/s 
     |████████████████████████████████| 1.6 MB 46.1 MB/s 
     |████████████████████████████████| 5.8 MB 54.0 MB/s 
     |████████████████████████████████| 438 kB 41.7 MB/s 
  Attempting uninstall: gast
    Found existing installation: gast 0.5.3
    Uninstalling gast-0.5.3:
      Successfully uninstalled gast-0.5.3
  Attempting uninstall: flatbuffers
    Found existing installation: flatbuffers 2.0
    Uninstalling flatbuffers-2.0:
      Successfully uninstalled flatbuffers-2.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.8.0 requires tf-estimator-nightly==2.8.0.dev2021122109, but you have tf-estimator-nightly 2.10.0.dev2022042708 which is incompatible.
2.10.0-dev20220427


In [2]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
!wget https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv

train_file_path = "train-data.tsv"
test_file_path = "valid-data.tsv"

--2022-04-27 13:10:11--  https://cdn.freecodecamp.org/project-data/sms/train-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358233 (350K) [text/tab-separated-values]
Saving to: ‘train-data.tsv’

train-data.tsv      100%[===================>] 349.84K  --.-KB/s    in 0.01s   

2022-04-27 13:10:11 (29.8 MB/s) - ‘train-data.tsv’ saved [358233/358233]

--2022-04-27 13:10:11--  https://cdn.freecodecamp.org/project-data/sms/valid-data.tsv
Resolving cdn.freecodecamp.org (cdn.freecodecamp.org)... 172.67.70.149, 104.26.3.33, 104.26.2.33, ...
Connecting to cdn.freecodecamp.org (cdn.freecodecamp.org)|172.67.70.149|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 118774 (116K) [text/tab-separated-values]
Saving to: ‘valid-data.tsv’

valid-data.tsv      100%[==========

In [3]:
# read the files into pandas dataframes and display them

train_df = pd.read_csv(train_file_path, header=None, sep='\t')
test_df = pd.read_csv(test_file_path, header=None, sep='\t')
display(train_df)
display(test_df)

# convert the dataframes into tensorflow datasets, replacing 'ham' and
# 'spam' with 0 and 1 respectively.

train_features = train_df[1]
train_labels = train_df[0].map({'ham': 0, 'spam': 1})
test_features = test_df[1]
test_labels = test_df[0].map({'ham': 0, 'spam': 1})

train_dataset = tf.data.Dataset.from_tensor_slices((train_features, train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((test_features, test_labels))

# print the first two elements of each dataset

for x, y in train_dataset.take(2):
  print('message: ', x.numpy())
  print('label: ', y.numpy())

for x, y in test_dataset.take(2):
  print('message: ', x.numpy())
  print('label: ', y.numpy())

,0,1
0,ham,ahhhh...just woken up!had a bad dream about u ...
1,ham,you can never do nothing
2,ham,"now u sound like manky scouse boy steve,like! ..."
3,ham,mum say we wan to go then go... then she can s...
4,ham,never y lei... i v lazy... got wat? dat day ü ...
...,...,...
4174,ham,just woke up. yeesh its late. but i didn't fal...
4175,ham,what do u reckon as need 2 arrange transport i...
4176,spam,free entry into our £250 weekly competition ju...
4177,spam,-pls stop bootydelious (32/f) is inviting you ...


,0,1
0,ham,i am in hospital da. . i will return home in e...
1,ham,"not much, just some textin'. how bout you?"
2,ham,i probably won't eat at all today. i think i'm...
3,ham,don‘t give a flying monkeys wot they think and...
4,ham,who are you seeing?
...,...,...
1387,ham,true dear..i sat to pray evening and felt so.s...
1388,ham,"what will we do in the shower, baby?"
1389,ham,where are you ? what are you doing ? are yuou ...
1390,spam,ur cash-balance is currently 500 pounds - to m...


message:  b'ahhhh...just woken up!had a bad dream about u tho,so i dont like u right now :) i didnt know anything about comedy night but i guess im up for it.'
label:  0
message:  b'you can never do nothing'
label:  0
message:  b'i am in hospital da. . i will return home in evening'
label:  0
message:  b"not much, just some textin'. how bout you?"
label:  0


In [4]:
# preprocessing
# batch and prefetch the data, the train_dataset will also be shuffled

# guaranteed perfect shuffling
BUFFER_SIZE = len(train_dataset)                                                
BATCH_SIZE = 64

train_dataset = train_dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# encoding the data. Create an encoder layer and adapt it to the train_dataset.
# The layer maps text to a vector of integers. Each integer corresponds to a
# "word", 0 and 1 correspond to the padding and the out of vocabulary words
# respectively.
# The other integers correspond to the words ordered by decreasing frequency
# in the dataset.

encoder = tf.keras.layers.TextVectorization(max_tokens=None)
encoder.adapt(train_dataset.map(lambda text, label: text))

# the encoder layer will pad each input to the max of the lengths of the
# inputs for each batch, so we do not pad nor truncate the data while
# preprocessing.

In [5]:
# Look at the first 15 words in the encoder vocabulary

vocabulary = encoder.get_vocabulary()
print(vocabulary[:15])

['', '[UNK]', 'to', 'i', 'you', 'a', 'the', 'u', 'and', 'in', 'is', 'me', 'my', 'for', 'your']


In [6]:
# creating the model

model = tf.keras.Sequential([
    encoder,
    tf.keras.layers.Embedding(
        # the input dimension in the embedding layer is the number of
        # words in the encoding vocabulary.
        input_dim=len(encoder.get_vocabulary()),                                
        output_dim=64,
        # mask_zero=True makes the model ignore input timesteps
        # corresponding to the padding                                            
        mask_zero=True),                                                        
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

# display a summary of the model

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_vectorization (TextVec  (None, None)             0         
 torization)                                                     
                                                                 
 embedding (Embedding)       (None, None, 64)          523584    
                                                                 
 lstm (LSTM)                 (None, 64)                33024     
                                                                 
 dense (Dense)               (None, 64)                4160      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 560,833
Trainable params: 560,833
Non-trainable params: 0
__________________________________________________

In [7]:
# compile the model

model.compile(loss="binary_crossentropy",
              optimizer="adam",metrics=['acc'])

In [8]:
# train the model

NUM_EPOCHS = 2
# two epochs give enough validation accuracy and avoid overfitting
history = model.fit(train_dataset,
                    epochs=NUM_EPOCHS, validation_data=test_dataset)

Epoch 1/2
66/66 [==============================] - 23s 215ms/step - loss: 0.3612 - acc: 0.8950 - val_loss: 0.0823 - val_acc: 0.9784
Epoch 2/2
66/66 [==============================] - 10s 145ms/step - loss: 0.0449 - acc: 0.9899 - val_loss: 0.0407 - val_acc: 0.9871


In [9]:
# function to predict messages based on model
# (should return list containing prediction and label, ex. [0.008318834938108921, 'ham'])
def predict_message(pred_text):
  probability = float(model.predict([pred_text], verbose=0)[0])
  # we use a classification threshold of 0.5
  label = 'ham' if probability < 0.5 else 'spam'                                
  prediction = [probability, label]
  return (prediction)

pred_text = "how are you doing today?"

prediction = predict_message(pred_text)
print(prediction)

[0.00034577446058392525, 'ham']


In [10]:
# Run this cell to test your function and model. Do not modify contents.
def test_predictions():
  test_messages = ["how are you doing today",
                   "sale today! to stop texts call 98912460324",
                   "i dont want to go. can we try it a different day? available sat",
                   "our new mobile video service is live. just install on your phone to start watching.",
                   "you have won £1000 cash! call to claim your prize.",
                   "i'll bring it tomorrow. don't forget the milk.",
                   "wow, is your arm alright. that happened to me one time too"
                  ]

  test_answers = ["ham", "spam", "ham", "spam", "spam", "ham", "ham"]
  passed = True

  for msg, ans in zip(test_messages, test_answers):
    prediction = predict_message(msg)
    if prediction[1] != ans:
      passed = False

  if passed:
    print("You passed the challenge. Great job!")
  else:
    print("You haven't passed yet. Keep trying.")

test_predictions()


You passed the challenge. Great job!
